In [ ]:
import numpy as np
from loguru import logger
from glob import glob
import os
import os.path as osp
import mmcv
from avcv.coco import *

import argparse

datasets = [
    ('/data/DMS_Behavior_Detection/Action_Eating_frames/annotations/instances_default.json', 
        '/data/DMS_Behavior_Detection/Action_Eating_frames/images', 'behavior'),
    ('/data/DMS_Behavior_Detection/Mobile_Cigarret_DB/mobile_cigarette_train_081522_finetuning.json', 
        '/data/DMS_Behavior_Detection/Mobile_Cigarret_DB/', 'behavior'),
    ('/data/DMS_Behavior_Detection/Mobile_Cigarret_DB/mobile_cigarette_val_080422.json', 
        '/data/DMS_Behavior_Detection/Mobile_Cigarret_DB/', 'behavior'),
    ('/data/public/coco/annotations/instances_train2017.json', '/data/public/coco/train2017/', 'behavior'),
    ('/data/public/coco/annotations/instances_val2017.json', '/data/public/coco/val2017/', 'behavior'),
    # ('/data/full-version-vip-pro/coco_annotations/train.json', '/data/full-version-vip-pro/DMS_DB_090922/', 'face')
]
new_root_dir = '/data/DMS_Behavior_Detection/merge-phone-cigaret-food/'

mmcv.mkdir_or_exist(new_root_dir)

cats = []
for ann, _,_ in datasets:
    for cat in mmcv.load(ann)['categories']:
        cats.append(cat['name'])

print(sorted(set(cats)))

['cam', 'cigarette', 'face', 'face_visible', 'food/drink', 'hand', 'left eye close', 'left eye na', 'left eye open', 'lomui', 'longmayphai', 'longmaytrai', 'mask', 'matphai', 'mattrai', 'moingoai', 'moitrong', 'mouth', 'mouth close', 'mouth open', 'phone', 'right eye close', 'right eye na', 'right eye open', 'songmui', 'tammatphai', 'tammattrai']


In [ ]:
category_old_to_new = {
    'cigarette':'cigarette',
    'food/drink':'food/drink',
    'phone':'phone',
    # 'cell phone': 'phone',
    # 'donut':'food/drink',
    # 'hot dog':'food/drink',
    # 'pizza':'food/drink',
    # 'wine glass':'food/drink',
    # 'orange':'food/drink',
    # 'cake':'food/drink',
    # 'banana':'food/drink',
    # 'sandwich':'food/drink',
    # 'apple':'food/drink',
    'left_eye':'left_eye',
    'right_eye':'right_eye',
    'mouth':'mouth',
    'face':'face'
}
categories = []
used_names = set()
for id, name in enumerate(category_old_to_new.values()):
    if not name in used_names:
        categories += [{'id':id+1, 'name':name} ]
        used_names.add(name)
    
name2cat = {cat['name']:cat for cat in categories}

In [ ]:
[_['name'] for _ in categories]

['cigarette', 'food/drink', 'phone', 'left_eye', 'right_eye', 'mouth', 'face']

In [ ]:

from avcv.all import *


out_dict = dict(
    images=[],
    annotations=[],
    categories=categories

)

new_img_dir = osp.join(new_root_dir, 'images')
for json, img_dir, dname in datasets:
    cc = CocoDataset(json, img_dir)
    # cc.visualize();
    for img_id in cc.img_ids:
        img = cc.gt.imgs[img_id].copy()
        new_img_id = len(out_dict['images'])
        new_img = img.copy()
        new_img['id'] = new_img_id
        new_img['dataset_name'] = dname
        file_name = os.path.abspath(osp.join(cc.img_dir, img['file_name']))
        rel_path = os.path.relpath(file_name, new_img_dir)
        new_img['file_name'] = rel_path
        

        take = False
        for ann in cc.gt.imgToAnns[img_id]:
            ann = ann.copy()
            ann['image_id'] = new_img_id
            old_cat = cc.gt.cats[ann['category_id']]
            if old_cat['name'] in category_old_to_new:
                new_cat_name = category_old_to_new[old_cat['name']]
                new_cat = name2cat[new_cat_name]
                ann['category_id'] = new_cat['id']
                ann['id'] = len(out_dict['annotations'])
                ann['area'] = ann['bbox'][2]*ann['bbox'][3]
                out_dict['annotations'].append(ann)
                take = True
        if not '/public/coco' in new_img['file_name'] or take:
            if not 'width' in new_img:
                new_img['width'], new_img['height'] = Image.open(file_name).size
                
            out_dict['images'].append(new_img)
        # if get_name(file_name) == 'vietanh_phoneusage_0012':
        #     vis = cc.visualize(img_id)
        #     new_cc = CocoDataset(out_dict, new_img_dir)
        #     new_vis = new_cc.visualize(new_img_id)

        #     mmcv.imwrite(np.concatenate([vis, new_vis], 1), 'vis.jpg')
            # import ipdb; ipdb.set_trace()


train_coco, val_coco = split_coco(out_dict)


2022-09-16 10:55:18.893 | INFO     | avcv.coco:split_coco:470 - Spliting ratio 0.85 -> 12739 training and 2249 testing


In [ ]:
for ann in out_dict['annotations']:
    ann['area'] = ann['bbox'][2]*ann['bbox'][3]

In [ ]:
train_coco, val_coco = split_coco(out_dict)


2022-09-16 10:55:19.018 | INFO     | avcv.coco:split_coco:470 - Spliting ratio 0.85 -> 12739 training and 2249 testing


In [ ]:
len(train_coco['images']), len(val_coco['images'])

(12739, 2249)

In [ ]:
train_json_out = osp.join(new_root_dir, 'annotations/food_face_train.json')
val_json_out = osp.join(new_root_dir, 'annotations/food_face_val.json')
mmcv.dump(train_coco, train_json_out)
mmcv.dump(val_coco, val_json_out)

In [ ]:
cc = CocoDataset(train_json_out, '/data/DMS_Behavior_Detection/merge-phone-cigaret-food/images/')

In [ ]:
# cc.img_dir

In [ ]:
# cc.visualize(show=1);

In [ ]:
# img2cats = {img_id:set() for img_id in cc.img_ids}
# for img_id in cc.img_ids:
#     anns = cc.gt.imgToAnns[img_id]
#     for ann in anns:
#         img2cats[img_id].add(ann['category_id'])

In [ ]:
pd.DataFrame.from_dict(img2cats, orient='index')

,0,1,2,3
9884,NaN,NaN,NaN,NaN
37683,16.0,13.0,14.0,15.0
38292,16.0,13.0,14.0,15.0
12937,NaN,NaN,NaN,NaN
16069,1.0,NaN,NaN,NaN
...,...,...,...,...
12946,NaN,NaN,NaN,NaN
24163,1.0,NaN,NaN,NaN
18080,1.0,NaN,NaN,NaN
38341,16.0,13.0,14.0,15.0
